### Imports

In [1]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 186 µs (started: 2023-06-09 09:41:36 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp5
Method:		 dgi
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
Option:		 
Subgroups id op: {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 2.93 ms (started: 2023-06-09 09:41:36 -05:00)


### Edge embeddings

In [3]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

 33%|███▎      | 1/3 [17:30<35:01, 1050.52s/it]

In [ ]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0         1         2
0 1  6.461881  0.000695  0.480581
  2  0.003822  0.842727  3.424330
  3  0.783019  0.024875  0.354545
  4  2.682043  2.122767  2.174131
  5  0.445657  1.257031  0.000274

time: 496 ms (started: 2023-06-09 07:59:14 -05:00)


### Concat edge embeddings

In [ ]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 3/3 [02:59<00:00, 59.78s/it]

time: 2min 59s (started: 2023-06-09 07:59:15 -05:00)


In [ ]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0         1         2  subgroup
0 1  6.461881  0.000695  0.480581         0
  2  0.003822  0.842727  3.424330         0
  3  0.783019  0.024875  0.354545         0
  4  2.682043  2.122767  2.174131         0
  5  0.445657  1.257031  0.000274         0

time: 4.68 s (started: 2023-06-09 08:02:14 -05:00)


In [ ]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.close()

100%|██████████| 3/3 [14:54<00:00, 298.09s/it]

time: 14min 54s (started: 2023-06-09 08:02:19 -05:00)


### Outliers detection

In [ ]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 3.64 ms (started: 2023-06-09 08:17:13 -05:00)


In [ ]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [01:00<00:00, 20.28s/it]

time: 1min (started: 2023-06-09 08:17:14 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
0 3    0.783019  0.024875  0.354545         0
  6    0.000378  0.239430  0.066928         0
  7    2.496010  0.103704  0.548030         0
  10   0.492603  0.250157  0.101900         0
1 131  2.037267  0.176170  0.041174         0

time: 12.5 ms (started: 2023-06-09 08:18:15 -05:00)


In [ ]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.close()

  0%|          | 0/3 [00:00<?, ?it/s]

Total: 12232238
Num. of inliers: 11009014
Num. of inliers: 1223224


 33%|███▎      | 1/3 [04:59<09:58, 299.35s/it]

Total: 17089033
Num. of inliers: 15380129
Num. of inliers: 1708904


 67%|██████▋   | 2/3 [11:59<06:10, 370.65s/it]

Total: 6044291
Num. of inliers: 5439862
Num. of inliers: 604429


100%|██████████| 3/3 [14:27<00:00, 289.19s/it]

time: 14min 27s (started: 2023-06-09 08:18:15 -05:00)


###  Filter common edges

In [ ]:
print(subgroups_id)
print(subgroups_id_op)


{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 736 µs (started: 2023-06-09 08:32:42 -05:00)


In [ ]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

11009014it [04:53, 37543.78it/s]it/s]
15380129it [06:49, 37516.62it/s]3, 301.77s/it]
5439862it [02:26, 37147.44it/s]12, 372.04s/it]
100%|██████████| 3/3 [14:34<00:00, 291.52s/it]

time: 14min 34s (started: 2023-06-09 08:32:42 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
59.0049  369.1213  0.783019  0.024875  0.354545         0
         503.0807  0.000378  0.239430  0.066928         0
         558.4273  2.496010  0.103704  0.548030         0
         977.8237  0.492603  0.250157  0.101900         0
274.0123 191.046   2.037267  0.176170  0.041174         0

time: 11.9 ms (started: 2023-06-09 08:47:17 -05:00)


In [ ]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

time: 619 µs (started: 2023-06-09 08:47:17 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
59.0049  369.1213  0.783019  0.024875  0.354545         0
         503.0807  0.000378  0.239430  0.066928         0
         558.4273  2.496010  0.103704  0.548030         0
         977.8237  0.492603  0.250157  0.101900         0
274.0123 191.046   2.037267  0.176170  0.041174         0

time: 7.19 ms (started: 2023-06-09 08:47:17 -05:00)


In [ ]:
df_edge_embeddings_concat_filter.index[:5]

MultiIndex([( '59.0049', '369.1213'),
            ( '59.0049', '503.0807'),
            ( '59.0049', '558.4273'),
            ( '59.0049', '977.8237'),
            ('274.0123',  '191.046')],
           )

time: 4.1 ms (started: 2023-06-09 08:47:18 -05:00)


In [ ]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

time: 1.08 ms (started: 2023-06-09 08:47:18 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(11009014, 4)


0         1         2  subgroup
59.0049  369.1213  0.783019  0.024875  0.354545         0
         503.0807  0.000378  0.239430  0.066928         0
         558.4273  2.496010  0.103704  0.548030         0
         977.8237  0.492603  0.250157  0.101900         0
274.0123 191.046   2.037267  0.176170  0.041174         0

time: 8.51 ms (started: 2023-06-09 08:47:18 -05:00)


In [ ]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # count
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["level_0", "level_1"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]

    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:14<00:00,  4.82s/it]

time: 14.5 s (started: 2023-06-09 08:47:18 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,384.0997,460.0954
1,690.4982,799.4705
2,719.4862,820.5967
3,377.0857,481.1227
4,719.4862,819.5543


time: 8.66 ms (started: 2023-06-09 08:47:33 -05:00)


In [ ]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28840 entries, 0 to 28839
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  28840 non-null  object
 1   target  28840 non-null  object
dtypes: object(2)
memory usage: 675.9+ KB
time: 19.7 ms (started: 2023-06-09 08:47:33 -05:00)


In [ ]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 3/3 [00:00<00:00, 43.88it/s]

time: 71.9 ms (started: 2023-06-09 08:47:33 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,384.0997,460.0954
1,690.4982,799.4705
2,719.4862,820.5967
3,377.0857,481.1227
4,719.4862,819.5543


time: 8.92 ms (started: 2023-06-09 08:47:33 -05:00)


In [ ]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28840 entries, 0 to 28839
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  28840 non-null  string
 1   target  28840 non-null  string
dtypes: string(2)
memory usage: 675.9 KB
time: 17 ms (started: 2023-06-09 08:47:33 -05:00)


In [ ]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 3/3 [01:08<00:00, 22.98s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
25653,109.0407,114.056,0.684680,0.654750,0.527725,0.859966,0.823646
26195,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
26183,109.0407,131.0462,0.802158,0.721020,0.613603,0.872517,0.867231
26226,109.0407,132.086,0.764665,0.614573,0.603960,0.893537,0.878882
26213,109.0407,135.0311,0.725125,0.726942,0.835541,0.930601,0.916594


time: 1min 9s (started: 2023-06-09 08:47:33 -05:00)


In [ ]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
25653,109.0407,114.056,0.684680,0.654750,0.527725,0.859966,0.823646
26195,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
26183,109.0407,131.0462,0.802158,0.721020,0.613603,0.872517,0.867231
26226,109.0407,132.086,0.764665,0.614573,0.603960,0.893537,0.878882
26213,109.0407,135.0311,0.725125,0.726942,0.835541,0.930601,0.916594


time: 15.1 ms (started: 2023-06-09 08:48:42 -05:00)


### Filter by STD and average weight

In [ ]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

100%|██████████| 3/3 [00:13<00:00,  4.65s/it]


,source,target,weight
0,109.0407,114.056,0.710154
1,109.0407,127.0513,0.830812
2,109.0407,131.0462,0.775306
3,109.0407,132.086,0.751123
4,109.0407,135.0311,0.826961


time: 14 s (started: 2023-06-09 08:48:43 -05:00)


In [ ]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,109.0407,114.056,0.710154
1,109.0407,127.0513,0.830812
2,109.0407,131.0462,0.775306
3,109.0407,132.086,0.751123
4,109.0407,135.0311,0.826961


time: 30.3 ms (started: 2023-06-09 08:48:57 -05:00)


In [ ]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

  0%|          | 0/3 [00:00<?, ?it/s]

Group: WT
Num. nodes: 2305
Num. edges: 28617



 67%|██████▋   | 2/3 [00:01<00:00,  1.94it/s]

Group: zwf1^
Num. nodes: 4875
Num. edges: 495251



100%|██████████| 3/3 [00:01<00:00,  1.57it/s]

Group: pck1^
Num. nodes: 5276
Num. edges: 445248

time: 1.91 s (started: 2023-06-09 08:48:57 -05:00)
